<a href="https://colab.research.google.com/github/GustavoHBDuarte/purchase_propensity_R/blob/development/purchase_propensity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTE: This notebook is unnecessary since Google Colab now directly supports R. Please navigate to `Runtime -> Change runtime type` and select `R`.

In [ ]:
cat(system('python3 -c "from google.colab import drive\ndrive.mount()"', intern=TRUE), sep='\n', wait=TRUE)

Warning message in system("python3 -c \"from google.colab import drive\ndrive.mount()\"", :
“running command 'python3 -c "from google.colab import drive
drive.mount()"' had status 1”



TRUE


In [ ]:
?system

In [ ]:
install.packages("rgl", repos = "http://cran.rstudio.com/")
install.packages("ConsRank", repos = "http://cran.rstudio.com/")
library("ConsRank")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘miniUI’, ‘webshot’, ‘htmlwidgets’, ‘shiny’, ‘crosstalk’, ‘manipulateWidget’



In [ ]:
system("add-apt-repository -y ppa:marutter/rrutter")
system("add-apt-repository -y ppa:marutter/c2d4u")
system("apt-get update")
system("apt install -y r-cran-rstan")

In [ ]:
install.packages('ggmap')
install.packages('codetools')

In [ ]:
library("ggmap")

In [ ]:
options(repr.plot.width  = 3,
        repr.plot.height = 3)
Sys.setenv(USE_CXX14 = 1)
library("rstan") # observe startup messages
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

In [ ]:
x <- seq(0, 2*pi, length.out=50)
plot(x, sin(x))

In [ ]:
stanmodelcode <- "
data {
    int<lower=0> N;
    real y[N];
}

parameters {
    real mu;

}

model {
    target += normal_lpdf(mu | 0, 10);
    target += normal_lpdf(y | mu, 1);
}"

y <- rnorm(20)
dat <- list(N = 20, y = y);

fit <- stan(model_code = stanmodelcode, model_name = "example",
            data = dat, iter = 2012, chains = 8, sample_file = 'norm.csv',
            verbose = FALSE,
            refresh = -1)

print(fit)

In [ ]:
plot(fit)

# 0 - Starting point